In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import csv
import random

class SequenceMaskedTicTacToeDataset(Dataset):
    def __init__(self, csv_file, sequence_length=25, state_dim=(3, 3)):
        self.data = []
        self.state_dim = state_dim
        self.sequence_length = sequence_length
        
        with open(csv_file, 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  # Skip header
            current_sequence = []
            
            for row in csv_reader:
                agent = int(row[0])
                state = np.array([float(x) for x in row[1][1:-1].split()]).reshape(state_dim)
                current_sequence.append((agent, state))
                
                if len(current_sequence) == sequence_length:
                    self.data.append(current_sequence)
                    current_sequence = current_sequence[1:]  # Slide the window
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        
        # Choose a random position to mask
        # mask_position = random.randint(0, self.sequence_length - 1)
        mask_position = -1
        
        # Create masked version of the sequence
        masked_sequence = [state.copy() for _, state in sequence]
        original_sequence = [state for _, state in sequence]
        
        # Mask the chosen position
        mask = np.ones_like(masked_sequence[mask_position], dtype=bool)
        masked_sequence[mask_position] = np.full_like(masked_sequence[mask_position], -1)
        
        return (
            torch.tensor([agent for agent, _ in sequence], dtype=torch.long),
            torch.tensor(np.array(masked_sequence), dtype=torch.float),
            torch.tensor(np.array(original_sequence), dtype=torch.float),
            torch.tensor(mask, dtype=torch.bool),
            torch.tensor(mask_position, dtype=torch.long)
        )


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np

class SequenceTicTacToePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2):
        super(SequenceTicTacToePredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, 3, 3)
        batch_size, seq_len, rows, cols = x.shape
        x = x.float()  # Convert to float for LSTM processing
        x = x.view(batch_size, seq_len, -1)  # Flatten each state
        
        lstm_out, _ = self.lstm(x)
        predictions = self.fc(lstm_out)
        predictions = predictions.view(batch_size, seq_len, rows, cols)
        
        # Round predictions to nearest integer during forward pass
        rounded_predictions = torch.round(predictions)
        # Use straight-through estimator for backpropagation
        rounded_predictions = predictions + (rounded_predictions - predictions).detach()
        
        return rounded_predictions

In [5]:
class SequenceTicTacToePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2, dropout=0.2):
        super(SequenceTicTacToePredictor, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Adjust transformer input dimension to be divisible by num_heads
        transformer_dim = ((input_dim - 1) // 4 + 1) * 4  # Round up to nearest multiple of 4
        self.input_projection = nn.Linear(input_dim, transformer_dim)
        
        # LSTM branch
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # GRU branch
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        
        # Transformer branch
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=transformer_dim, nhead=4, dim_feedforward=hidden_dim, dropout=dropout),
            num_layers=num_layers
        )
        
        # CNN branch
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        
        # Shared dense layer
        self.shared_dense = nn.Linear(hidden_dim * 4, hidden_dim)
        
        # Output layers
        self.next_state = nn.Linear(hidden_dim, input_dim)
        self.agent = nn.Linear(hidden_dim, 2)  # Assuming 2 agents (1 and 2)
        self.context = nn.Linear(hidden_dim, input_dim)  # Context size same as input_dim
        self.anomaly = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        batch_size, seq_len, _ = x.shape
        
        # LSTM branch
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Take the last output
        
        # GRU branch
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:, -1, :]  # Take the last output
        
        # Transformer branch
        transformer_input = self.input_projection(x)
        transformer_out = self.transformer(transformer_input.transpose(0, 1)).transpose(0, 1)
        transformer_out = transformer_out[:, -1, :]  # Take the last output
        
        # CNN branch
        cnn_out = self.cnn(x.transpose(1, 2)).squeeze(-1)
        
        # Concatenate all branch outputs
        combined = torch.cat([lstm_out, gru_out, transformer_out, cnn_out], dim=1)
        
        # Shared dense layer
        shared = self.relu(self.shared_dense(combined))
        shared = self.dropout(shared)
        
        # Output layers
        next_state = self.next_state(shared)
        agent = self.agent(shared)
        context = self.context(shared)
        anomaly = torch.sigmoid(self.anomaly(shared))
        
        return next_state, agent, context, anomaly

In [7]:

# Hyperparameters
input_dim = 9  # 3x3 grid flattened
hidden_dim = 64
num_layers = 2
dropout = 0.2
num_epochs = 300

# Create the model
model = SequenceTicTacToePredictor(input_dim, hidden_dim, num_layers, dropout)

# Define loss functions and optimizer
next_state_criterion = nn.MSELoss()
agent_criterion = nn.CrossEntropyLoss()
context_criterion = nn.MSELoss()
anomaly_criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop (pseudo-code)
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        
        x, next_state_target, agent_target, context_target, anomaly_target = batch
        
        next_state_pred, agent_pred, context_pred, anomaly_pred = model(x)
        
        next_state_loss = next_state_criterion(next_state_pred, next_state_target)
        agent_loss = agent_criterion(agent_pred, agent_target)
        context_loss = context_criterion(context_pred, context_target)
        anomaly_loss = anomaly_criterion(anomaly_pred, anomaly_target)
        
        total_loss = next_state_loss + agent_loss + context_loss + 0.1 * anomaly_loss
        
        total_loss.backward()
        optimizer.step()

c:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


NameError: name 'dataloader' is not defined